In [1]:
# Mount Google Drive (required if running on Google Colab)
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Install required modules
# !pip install '/content/drive/Team Drives/Deep Learning Team Drive/yelp_dataset_challenge-master/yelp_util'
!pip install Cython
# !pip install word2vec
!pip install gensim
!pip install unidecode
!pip install textblob
!pip install wordcloud


# Import required modules
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# import yelp_util
# from yelp_util import downloader as dl
import os
from scipy import misc
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import pandas as pd
import random
from wordcloud import WordCloud
import matplotlib.pyplot as plt

    100% |████████████████████████████████| 2.1MB 12.7MB/s 
    100% |████████████████████████████████| 23.6MB 1.9MB/s 
    100% |████████████████████████████████| 1.4MB 16.1MB/s 
    100% |████████████████████████████████| 133kB 26.8MB/s 
    100% |████████████████████████████████| 61kB 23.0MB/s 
    100% |████████████████████████████████| 5.1MB 7.2MB/s 
    100% |████████████████████████████████| 552kB 22.1MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
    100% |████████████████████████████████| 245kB 7.8MB/s 
    100% |████████████████████████████████| 645kB 20.9MB/s 
    100% |████████████████████████████████| 368kB 26.7MB/s 
[nltk_data] Downloading package 

In [4]:
# Download and unpickle data

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'

#Chinese Restaurant Reviews
#review = pd.read_pickle(os.path.join(download_path, 'chinese_reviews.pickle'))
#review = pd.read_pickle(os.path.join(download_path, 'italian_reviews.pickle'))


reviewItaly = pd.read_pickle(os.path.join(download_path, 'italian_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=2018).permutation(reviewItaly.shape[0])

# Reset index so it stays in the new order
reviewItaly = reviewItaly.iloc[shuffidx].reset_index(drop=True)
reviewItaly.head()

,business_id,date,review_id,stars,text,type,user_id,votes_cool,votes_funny,votes_useful
0,FwYIyHrdkkB28wbbEDJ9Hg,2014-08-13,5baYd00l1HN7OiDdnYsVmg,4,"Ohhhh, I've put off writing this review foreve...",review,fzy5CCuLns6n9IUTsdTI8w,2,2,3
1,DnkoH96JRzX9udAPLHBbIQ,2014-01-07,TnFknNnvzB55SRyMjtn1zw,5,Wonderful great tasting food no fancy flair JU...,review,K7MW3SInIPA6uVV3E8rxMA,1,0,1
2,PayaNmMeZFOFeS_jo7eB8g,2014-07-01,ApbXgWsyJ0Ue933EJDl_8Q,5,"This place is the best! So far, it is hands do...",review,yUtqMCrl86TkLXKjGsXQ0g,0,0,0
3,79EG9Pv0bxNTmBjTVCd4CQ,2012-01-05,C_uOMbJ88EQHA_1LzGa3lw,3,The experience was a bit rocky. It was nice t...,review,DdPGlglwpwbLaRFTwAL2tQ,0,0,0
4,m9K8o-paI6w500w9P8G9pg,2010-05-02,OT2toKtJzAABpmaC4s0edw,4,Hey mambo! mambo italiano! Go Go Joe! Now this...,review,XWvowG_OOBd2g3EjNGUtOQ,3,2,5


In [0]:
#PREOPROCESS A LITTLE BIT

#review2 = review.loc[:, ['stars','text']]
#review2.head()
#review3 = review.loc[:, ['stars','text']]
reviewItaly = reviewItaly.loc[:, ['stars','text']]

In [7]:
#Option 1: review2 makes stars > 3 = positive, and the rest negative

#review2['sentiment']=['pos' if (x>3) else 'neg' for x in review2['stars']]
#review2.head()

#Option 2: review3 makes stars > 3 = positive, and stars = 2 or 1 negative. 3 are neutral/not considered for analysis.

#get rid of rows with 3 stars
#review3 = review3[review3.stars != 3]
#review3.stars.value_counts() # check that there are no "3" stars
#review3['sentiment']=['pos' if (x>3) else 'neg' for x in review3['stars']] #add the sentiment column
#review3.head()
reviewItaly = reviewItaly[reviewItaly.stars != 3]
reviewItaly.stars.value_counts() # check that there are no "3" stars
reviewItaly['sentiment']=['pos' if (x>3) else 'neg' for x in reviewItaly['stars']] #add the sentiment column
reviewItaly.head()


,stars,text,sentiment
0,4,"Ohhhh, I've put off writing this review foreve...",pos
1,5,Wonderful great tasting food no fancy flair JU...,pos
2,5,"This place is the best! So far, it is hands do...",pos
4,4,Hey mambo! mambo italiano! Go Go Joe! Now this...,pos
5,4,"I've been eating so much pizza lately, I think...",pos


In [8]:

#lowercase/remove punctuations

import re
#review2['text']= [x.lower() for x in review2['text']]
#review2['text'] = review2['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
#review2.head()

reviewItaly['text']= [x.lower() for x in reviewItaly['text']]
reviewItaly['text'] = reviewItaly['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
reviewItaly.head()

,stars,text,sentiment
0,4,ohhhh ive put off writing this review forever ...,pos
1,5,wonderful great tasting food no fancy flair ju...,pos
2,5,this place is the best so far it is hands down...,pos
4,4,hey mambo mambo italiano go go joe now this is...,pos
5,4,ive been eating so much pizza lately i think i...,pos


In [9]:
reviewItaly.sentiment.value_counts()

pos    56582
neg    15293
Name: sentiment, dtype: int64

In [10]:
#################
#### L S T M ####
##### starts ####
##### here ###### 
#################

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, CuDNNLSTM, Bidirectional
from keras.metrics import categorical_accuracy
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

tokenizer = Tokenizer(num_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',                                  
                      lower=True,split=' ')
#tokenizer.fit_on_texts(review2['text'].values)
tokenizer.fit_on_texts(reviewItaly['text'].values)

Using TensorFlow backend.


In [11]:
#Use Tokenizer to vectorize the text and convert it into sequence of integers 
tokenizer

In [0]:
#X = tokenizer.texts_to_sequences(review2['text'].values)
#X = tokenizer.texts_to_sequences(review3['text'].values)
X = tokenizer.texts_to_sequences(reviewItaly['text'].values)
X = pad_sequences(X)

In [13]:
#len(review2) #output: 51984
#len(review3) #output: 43017
len(reviewItaly)

71875

In [51]:
### The following sections ###
### were exectued with the following hyper parameters ###
#embed_dim = 128
#lstm_out = 200
#batch_size = 32
#dropout layer
#vocabulary size = 2000

#activation function - softmax
#optimized = adam
#uses CuDNNLSTM !!!!! (GPU powered)


embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(CuDNNLSTM(lstm_out, return_sequences=False, input_shape=(869, 128)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 869, 128)          256000    
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 869, 128)          0         
_________________________________________________________________
cu_dnnlstm_6 (CuDNNLSTM)     (None, 200)               264000    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 402       
Total params: 520,402
Trainable params: 520,402
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
#Y = pd.get_dummies(review2['sentiment']).values
Y = pd.get_dummies(reviewItaly['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(57500, 932) (57500, 2)
(14375, 932) (14375, 2)


In [16]:
#Fit model
model.fit(X, Y, validation_split=0.1, epochs=2, batch_size=batch_size, verbose=2)

Train on 64687 samples, validate on 7188 samples
Epoch 1/2
 - 376s - loss: 0.2166 - acc: 0.9164 - val_loss: 0.1502 - val_acc: 0.9442
Epoch 2/2
 - 372s - loss: 0.1419 - acc: 0.9470 - val_loss: 0.1299 - val_acc: 0.9534


In [17]:
#score: evaluation of the loss function
#loss, TEST accuracy
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.10
Validation Accuracy: 0.96


In [21]:
##MEXICAN RESTAURANT REVIEWS##

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'
reviewMex = pd.read_pickle(os.path.join(download_path, 'mexican_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=2018).permutation(reviewMex.shape[0])

# Preprocess
reviewMex = reviewMex.iloc[shuffidx].reset_index(drop=True)
reviewMex = reviewMex.loc[:, ['stars','text']]
reviewMex = reviewMex[reviewMex.stars != 3]
reviewMex.stars.value_counts() # check that there are no "3" stars
reviewMex['sentiment']=['pos' if (x>3) else 'neg' for x in reviewMex['stars']] #add the sentiment column
reviewMex.head()

#lowercase/remove punctuations

import re

reviewMex['text']= [x.lower() for x in reviewMex['text']]
reviewMex['text'] = reviewMex['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
reviewMex.head()

,stars,text,sentiment
0,5,amazing cheese enchiladas the beans are fantas...,pos
1,5,the place is great the food is even better if ...,pos
2,5,i love this place for a quick healthy i always...,pos
3,4,its possible that i was just really hungry i...,pos
4,5,great cheap eats here they have a 3 soft tacos...,pos


In [0]:
tokenizer.fit_on_texts(reviewMex['text'].values)

In [24]:
X = tokenizer.texts_to_sequences(reviewMex['text'].values)
X = pad_sequences(X)
Y = pd.get_dummies(reviewMex['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(64582, 906) (64582, 2)
(16146, 906) (16146, 2)


In [27]:
#Fit model
model.fit(X, Y, validation_split=0.1, epochs=2, batch_size=batch_size, verbose=2)

Train on 72655 samples, validate on 8073 samples
Epoch 1/2
 - 408s - loss: 0.2269 - acc: 0.9105 - val_loss: 0.1564 - val_acc: 0.9415
Epoch 2/2
 - 407s - loss: 0.1639 - acc: 0.9364 - val_loss: 0.1375 - val_acc: 0.9491


In [29]:
#score: evaluation of the loss function
#loss, TEST accuracy
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.11
Validation Accuracy: 0.9576


In [30]:
##HAIR SALON REVIEWS##

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'
reviewHair = pd.read_pickle(os.path.join(download_path, 'salon_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=2018).permutation(reviewHair.shape[0])

# Preprocess
reviewHair = reviewHair.iloc[shuffidx].reset_index(drop=True)
reviewHair = reviewHair.loc[:, ['stars','text']]
reviewHair = reviewHair[reviewHair.stars != 3]
reviewHair.stars.value_counts() # check that there are no "3" stars
reviewHair['sentiment']=['pos' if (x>3) else 'neg' for x in reviewHair['stars']] #add the sentiment column
reviewHair.head()

#lowercase/remove punctuations

import re

reviewHair['text']= [x.lower() for x in reviewHair['text']]
reviewHair['text'] = reviewHair['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
reviewHair.head()

,stars,text,sentiment
0,5,olivia gave me an awesome haircut she got me ...,pos
1,5,robert is a lifesaver no joke i got home late ...,pos
2,5,after my stylist and colorist left town i was...,pos
3,5,jai dcouvert aveda en arrivant au canada et ja...,pos
4,5,five of us were in town for a bachelorette par...,pos


In [31]:
tokenizer.fit_on_texts(reviewHair['text'].values)
X = tokenizer.texts_to_sequences(reviewHair['text'].values)
X = pad_sequences(X)
Y = pd.get_dummies(reviewHair['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(13218, 905) (13218, 2)
(3305, 905) (3305, 2)


In [34]:
#Fit model
model.fit(X, Y, validation_split=0.1, epochs=2, batch_size=batch_size, verbose=2)

Train on 14870 samples, validate on 1653 samples
Epoch 1/2
 - 84s - loss: 0.2252 - acc: 0.9117 - val_loss: 0.1441 - val_acc: 0.9468
Epoch 2/2
 - 83s - loss: 0.1295 - acc: 0.9504 - val_loss: 0.2084 - val_acc: 0.9123


In [35]:
#score: evaluation of the loss function
#loss, TEST accuracy
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.19
Validation Accuracy: 0.9207


In [36]:
##Home and Garden REVIEWS##

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'
reviewHome = pd.read_pickle(os.path.join(download_path, 'homegarden_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=2018).permutation(reviewHome.shape[0])

# Preprocess
reviewHome = reviewHome.iloc[shuffidx].reset_index(drop=True)
reviewHome = reviewHome.loc[:, ['stars','text']]
reviewHome = reviewHome[reviewHome.stars != 3]
reviewHome.stars.value_counts() # check that there are no "3" stars
reviewHome['sentiment']=['pos' if (x>3) else 'neg' for x in reviewHome['stars']] #add the sentiment column
reviewHome.head()

#lowercase/remove punctuations

import re

reviewHome['text']= [x.lower() for x in reviewHome['text']]
reviewHome['text'] = reviewHome['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
reviewHome.head()

,stars,text,sentiment
1,5,my wife and i were in phoenix for the weekend ...,pos
2,5,for home owners with pools we know how importa...,pos
3,5,cost plus makes me happy its near my work and...,pos
4,5,you dont find many pool professionals that are...,pos
6,5,before i moved cross country from phoenix to c...,pos


In [37]:
tokenizer.fit_on_texts(reviewHome['text'].values)
X = tokenizer.texts_to_sequences(reviewHome['text'].values)
X = pad_sequences(X)
Y = pd.get_dummies(reviewHome['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7288, 901) (7288, 2)
(1823, 901) (1823, 2)


In [40]:
#Fit model
model.fit(X, Y, validation_split=0.1, epochs=2, batch_size=batch_size, verbose=2)

Train on 8199 samples, validate on 912 samples
Epoch 1/2
 - 47s - loss: 0.4049 - acc: 0.8186 - val_loss: 0.3274 - val_acc: 0.8794
Epoch 2/2
 - 46s - loss: 0.2403 - acc: 0.9019 - val_loss: 0.2646 - val_acc: 0.8936


In [41]:
#score: evaluation of the loss function
#loss, TEST accuracy
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.19
Validation Accuracy: 0.9314


In [42]:
##Band and Credit Union REVIEWS##

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'
reviewBCU = pd.read_pickle(os.path.join(download_path, 'bankandcu_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=2018).permutation(reviewBCU.shape[0])

# Preprocess
reviewBCU = reviewBCU.iloc[shuffidx].reset_index(drop=True)
reviewBCU = reviewBCU.loc[:, ['stars','text']]
reviewBCU = reviewBCU[reviewBCU.stars != 3]
reviewBCU.stars.value_counts() # check that there are no "3" stars
reviewBCU['sentiment']=['pos' if (x>3) else 'neg' for x in reviewBCU['stars']] #add the sentiment column
reviewBCU.head()

#lowercase/remove punctuations

import re

reviewBCU['text']= [x.lower() for x in reviewBCU['text']]
reviewBCU['text'] = reviewBCU['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
reviewBCU.head()

,stars,text,sentiment
0,1,at citibank had a cd there for 5 years earned ...,neg
1,2,its not the people but how the bank itself run...,neg
2,5,just dropping by to say that even as ive moved...,pos
3,1,this is a review of us banks in general as a b...,neg
5,4,received great assistant from the new manager ...,pos


In [43]:
tokenizer.fit_on_texts(reviewBCU['text'].values)
X = tokenizer.texts_to_sequences(reviewBCU['text'].values)
X = pad_sequences(X)
Y = pd.get_dummies(reviewBCU['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1071, 817) (1071, 2)
(268, 817) (268, 2)


In [46]:
#Fit model
model.fit(X, Y, validation_split=0.1, epochs=2, batch_size=batch_size, verbose=2)

Train on 1205 samples, validate on 134 samples
Epoch 1/2
 - 7s - loss: 0.6731 - acc: 0.5925 - val_loss: 0.6698 - val_acc: 0.6194
Epoch 2/2
 - 6s - loss: 0.4512 - acc: 0.7917 - val_loss: 0.4090 - val_acc: 0.8507


In [47]:
#score: evaluation of the loss function
#loss, TEST accuracy
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.35
Validation Accuracy: 0.8507


In [48]:
##Chinese REVIEWS##

download_path = '/content/drive/Team Drives/Deep Learning Team Drive/data'
reviewChin = pd.read_pickle(os.path.join(download_path, 'chinese_reviews.pickle'))
# Suffle review data before any other preprocessing
shuffidx = np.random.RandomState(seed=2018).permutation(reviewChin.shape[0])

# Preprocess
reviewChin = reviewChin.iloc[shuffidx].reset_index(drop=True)
reviewChin = reviewChin.loc[:, ['stars','text']]
reviewChin = reviewChin[reviewChin.stars != 3]
reviewChin.stars.value_counts() # check that there are no "3" stars
reviewChin['sentiment']=['pos' if (x>3) else 'neg' for x in reviewChin['stars']] #add the sentiment column
reviewChin.head()

#lowercase/remove punctuations

import re

reviewChin['text']= [x.lower() for x in reviewChin['text']]
reviewChin['text'] = reviewChin['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
reviewChin.head()

,stars,text,sentiment
0,4,ive been to this place many times and it has c...,pos
1,4,chicago snowbird turned us on to this restaura...,pos
3,2,the item i wanted when i went thru the drive t...,neg
4,2,no i was not happy with my order its a new pl...,neg
7,4,tasty is accurate this is chinese for white pe...,pos


In [49]:
tokenizer.fit_on_texts(reviewChin['text'].values)
X = tokenizer.texts_to_sequences(reviewChin['text'].values)
X = pad_sequences(X)
Y = pd.get_dummies(reviewChin['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(34413, 869) (34413, 2)
(8604, 869) (8604, 2)


In [52]:
#Fit model
model.fit(X, Y, validation_split=0.1, epochs=2, batch_size=batch_size, verbose=2)

Train on 38715 samples, validate on 4302 samples
Epoch 1/2
 - 210s - loss: 0.3562 - acc: 0.8490 - val_loss: 0.2123 - val_acc: 0.9203
Epoch 2/2
 - 209s - loss: 0.1795 - acc: 0.9303 - val_loss: 0.1782 - val_acc: 0.9284


In [53]:
#score: evaluation of the loss function
#loss, TEST accuracy
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.15
Validation Accuracy: 0.9455
